# Requirements
The project requires all the followings:
1. **Executive Summary**
   
    Description of the selected project, problem to be solved, and basic description of the data set from the sources.

2. **Summary of the Project Context and Objectives**
   
    Summarise your project context and list the objectives covered in your project.

3. **Methodology**

    Create a data pipeline to depict your sequence of actions that move data from a source to a destination. Show and explain all phases or steps involved in the data mining process, including the data preprocessing (ETL/ ELT), modelling, evaluation and deployment in your project development.

4. **Results and Discussion**

    Produce and explain all codes, GUI, graphs, diagrams, or any visualisation output from the project development.

5. **Conclusion**

    Conclude your project and discuss some contributions to society, the environment, systems, etc.

6. **References**

    Cite every single reference used in completing the project. 


# Executive Summary

## Problem to be solved
In this competition's datasets, you are predicting the probability that an online transaction is fraudulent, as denoted by the binary target isFraud. 
For each TransactionID in the test set, you must predict a probability for the isFraud variable.

## Dataset Basic Description
Please note that in this report, we use the zip that include all dataset csv together. The data is broken into two files identity and transaction, which are joined by TransactionID. Not all transactions have corresponding identity information. As described by host,

Transaction Table *
- TransactionDT: timedelta from a given reference datetime (not an actual timestamp)
- TransactionAMT: transaction payment amount in USD
- ProductCD: product code, the product for each transaction
- card1 - card6: payment card information, such as card type, card category, issue bank, country, etc.
- addr: address
- dist: distance
- P_ and (R__) emaildomain: purchaser and recipient email domain
- C1-C14: counting, such as how many addresses are found to be associated with the payment card, etc. The actual meaning is masked.
- D1-D15: timedelta, such as days between previous transaction, etc.
- M1-M9: match, such as names on card and address, etc.
- Vxxx: Vesta engineered rich features, including ranking, counting, and other entity relations.

Categorical Features:
- ProductCD
- card1 - card6
- addr1, addr2
- P_emaildomain
- R_emaildomain
- M1 - M9

Identity Table *

Variables in this table are identity information – network connection information (IP, ISP, Proxy, etc) and digital signature (UA/browser/os/version, etc) associated with transactions.
They're collected by Vesta’s fraud protection system and digital security partners.
(The field names are masked and pairwise dictionary will not be provided for privacy protection and contract agreement)

Categorical Features:
- DeviceType
- DeviceInfo
- id_12 - id_38

## Additional Information about datasets
1. [Further Information and related discussion](https://www.kaggle.com/c/ieee-fraud-detection/discussion/101203)
2. [Kaggler's insight](https://www.kaggle.com/c/ieee-fraud-detection/discussion/101203#610146)
3. [Labelling logic](https://www.kaggle.com/c/ieee-fraud-detection/discussion/101203#589276)
    > Is this all real data, and if so, how confident are you that all fraud instances are correctly labeled? I ask because I see some suspicious rows that look like undetected fraud, or synthetic rows added to confuse one of the more obvious fraud detection algorithms.
    >
    >But then again, with such a large dataset one can expect to find some very improbable coincidences within it.


   > It's a good question.
   >
   > Yes, they're all real data, no synthetic data. The logic of our labeling is define reported chargeback on the card as fraud transaction (isFraud=1) and transactions posterior to it with either user account, email address or billing address directly linked to these attributes as fraud too. If none of above is reported and found beyond 120 days, then we define as legit transaction (isFraud=0).
    However, in real world fraudulent activity might not be reported, e.g. cardholder was unaware, or forgot to report in time and beyond the claim period, etc. In such cases, supposed fraud might be labeled as legit, but we never could know of them. Thus, we think they're unusual cases and negligible portion.
4. [Main Ideas from Grandmaster's EDA](https://www.kaggle.com/code/cdeotte/eda-for-columns-v-and-id/notebook)



# Summary of EDA

The main purpose of exploratory data analysis (EDA) determine strategies of the imputation, encoding, aggregration, reduction and transformation for each feature.

There are too much data to be fed into to later model development. Hence, feature selection is essential here. Feature selection techniques adopted are missing value analysis and correlation analysis.

Firstly, we group the features based on whether their missing values are happening together. This results multiple pages for example v1 until v11 are related via their missing values.

Since it is computationally infeasible to compute the correlation matrix of all 300++ vesta rich features, we compute correlation matrix of smaller pages that are found earlier. From that, we futher group the features based correlation value (i.e.: see `naive_reduce` function). Counting and temporal attributes are also grouped via this correlation selection approach. The grouped variables will be aggregrated into a sinlge variable in preprocessing step.

For identity dataset, we see that that identity info and fraudelent transaction are not strongly correlated (neither positve nor negative). Furthermore, not all identity info are provided, then we decide not to include them as input features for model developed.



# Methodology



The rationales behind the choices of preprocessing algorithms are made based on EDA. We adopt ELT steps that is extract, load and transform. 

## ELT Pipeline of Preprocessing 
We only apply few transformation at the data reading (e.g.: switching index, categorical typing and data enrichment), then the data will be fed into the preprocessing pipeline we have developed using sklearn library. The preprocessing pipeline only involves label encoding, missing value imputation, reduction via aggregration and numerical normalization.

> The order of the columns in the transformed feature matrix follows the order of how the columns are specified in the transformers list. Columns of the original feature matrix that are not specified are dropped from the resulting transformed feature matrix, unless specified in the passthrough keyword. Those columns specified with passthrough are added at the right to the output of the transformers.

In developing pipeline under `sklearn` framework, `ColumnTransformer` apply transformations parallelly whereas `Pipeline` apply transformations sequentially. Thus, we mix them to achive desired overall transformation. 

The results are a 2D array for features and a 1D array input target. The order of resultant feature array does not order as original data frame as mentioned by `sklearn` documentation. 

Using the results from EDA section, we will reduce features based on basis of correlation (i.e.: subsets group). To impute missing value and aggregate grouped variables, we have defined a basic unit constructor `impute_reduce_pipe`. `correlated_group_reducer` is built upon the `impute_reduce_pipe`.

Initially, we want the reducer adjust its imputation and aggregration strategies based on the grouped variables. For example, for one variable, we can skip; for 2 to 5, we can take the average; for more than 5, we do dimension reduction PCA. However, due to the size of dataset, KNN imputation and dimension reduction techniques don't scale well; too slow. Hence, we don't adopt such strategy.

As a result, we reduce 300++ features to 178 input features.

### Rationales of developing pipeline
Suppose that we've 10 input feature and 1 categorical target. The input feature is a random mix of categorical and continuous feature. We develop a code that impute and encode each feature accordingly and one-hot encode the categorical feature. We can use `ColumnTransformer` and `Pipeline` to describe such processing step so that the preprocessor receives and transform any data frame that has the same format. Furthermore, the preprocessor 'should' known how to inverse transform each encoded feature. Unfortunately, the complete inversible pipeline implementation is cumbersome and time consuming if using current `sklearn` framework.

## Modelling and Evaluation
> Champion's solution - https://www.kaggle.com/competitions/ieee-fraud-detection/discussion/111308

Refering to the competition champion's solution, XGB model perform very well. Therefore, we adopt the same model. Once we done model development and training, we submit the `submission.csv` to Kaggle viewing the percision score. 

Since the target feature distribution is heavily imbalanced, precision score is used rather than accuracy.

Trying different model with different parameters (e.g.: random forest and decision tree), the precision score is still capped at 0.70 whereas top 100 solutions all have above 0.95 score. The low percision score is due to we don't include identity dataset in model training. Also, we don't aggressively engineer our features via time consistency checking.

## Deployment

The `deployment.py` plot a windowed time series chart whose peak indicate a suspect transaction. It involves using `ELT.py`'s data generator to simulate the process of fetching data from other sources.